# Vk data preprocessing for further modelling

In [15]:
import pandas as pd
import numpy as np

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from pymystem3 import Mystem
from tqdm import tqdm_notebook

from string import punctuation

import sys
sys.path.append('../')
from src.utils import remove_name_from_text
from src.log_progress import log_progress

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
raw_data =  pd.read_csv('vk_posts.csv').loc[:, ['id', 'date', 'owner_id', 'text', 'likes', 'reposts', 'comments']]
raw_data.head()

,id,date,owner_id,text,likes,reposts,comments
0,66,1322907480,-1243876,"ГОЛОСУЕМ ЗА БОГУ!!\n\n""Дорогие друзья! Голосуй...",6,1,1
1,10588,1332165525,695803,"Позавчера я написал пост про то, как мы заехал...",8,3,16
2,800,1334033740,102648080,В эти минуты идет прямая трансляция лекции про...,2,1,0
3,5810,1336373356,70549719,12 апреля 2012 г. в Тюменском государственном ...,1,1,0
4,5003,1352718600,61866128,"Дорогие друзья, журнал ""ЕГОРКА"" участвует в ко...",2,1,2


In [3]:
print('Количество постов:', len(raw_data))

Количество постов: 74483


In [4]:
#vk_posts['date'] = pd.to_datetime(vk_posts['date'], unit='s')
#vk_posts = vk_posts.sort_values(by='date')
print('Количество дублирующихся постов:', raw_data.duplicated(subset='text').sum())

Количество дублирующихся постов: 17648


In [5]:
data = raw_data.drop_duplicates(subset='text').reset_index(drop=True)
print('Постов после удаления дубликатов:', len(data))

Постов после удаления дубликатов: 56835


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56835 entries, 0 to 56834
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        56835 non-null  int64 
 1   date      56835 non-null  int64 
 2   owner_id  56835 non-null  int64 
 3   text      56834 non-null  object
 4   likes     56835 non-null  int64 
 5   reposts   56835 non-null  int64 
 6   comments  56835 non-null  int64 
dtypes: int64(6), object(1)
memory usage: 2.8+ MB


In [7]:
data['date'] = pd.to_datetime(data['date'], unit='s')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56835 entries, 0 to 56834
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        56835 non-null  int64         
 1   date      56835 non-null  datetime64[ns]
 2   owner_id  56835 non-null  int64         
 3   text      56834 non-null  object        
 4   likes     56835 non-null  int64         
 5   reposts   56835 non-null  int64         
 6   comments  56835 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 2.8+ MB


In [8]:
data = data.dropna().reset_index(drop=True)
print('После удаления пустых значений осталось', len(data), 'записей')

После удаления пустых значений осталось 56834 записей


In [9]:
print('Количество дублирующихся строк:', data.duplicated().sum())

Количество дублирующихся строк: 0


In [10]:
data['txt_processed'] = data['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x.lower()))
data['txt_processed'] = data['txt_processed'].apply(lambda x: re.sub(r'\n+', ' ', x))
data['txt_processed'] = data['txt_processed'].apply(lambda x: re.sub(r'\d+', '', x))
data['txt_processed'] = data['txt_processed'].apply(lambda x: re.sub(r'id\w+', '', x))

data['txt_processed'] = data['txt_processed'].apply(lambda x: re.sub(r'\s+', ' ', x))
data['txt_processed'] = data['txt_processed'].apply(lambda x: x.strip())
data['txt_processed'].tail()

56829    ссылка на трансляцию сегодняшнего матча тюмень...
56830    евгения басакова рано поверили в победу в очер...
56831    анна поспелова была какаято паника приморочка ...
56832    приморочка провела первую в этом году встречу ...
56833    наталья васильченко нас подвели тонкости психо...
Name: txt_processed, dtype: object

In [11]:
lem = Mystem()
nltk.download('stopwords')
stop = stopwords.words('russian')

def preprocess_text(x):
    word_list = []
    for word in (''.join(lem.lemmatize(x))).split(' '):
        if word not in stop and word != '':
            word_list.append(word.rstrip())
    return word_list

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Зима\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
stop.extend([
u'я', u'а', u'да', u'но', u'тебе', u'мне', u'ты', u'и', u'у', u'на', u'ща', u'ага',
u'так', u'там', u'какие', u'который', u'какая', u'туда', u'давай', u'короче', u'кажется', u'вообще',
u'ну', u'не', u'чет', u'неа', u'свои', u'наше', u'наш', u'весь', u'хотя', u'такое', u'например', u'кароч', u'как-то',
u'нам', u'хм', u'всем', u'нет', u'да', u'оно', u'своем', u'про', u'вы', u'м', u'тд',
u'вся', u'кто-то', u'что-то', u'вам', u'это', u'эта', u'эти', u'этот', u'прям', u'либо', u'как', u'мы',
u'просто', u'блин', u'очень', u'самые', u'твоем', u'ваша', u'кстати', u'вроде', u'типа', u'пока', u'ок',
u'мочь'
])

In [13]:
text_data = data['txt_processed'].to_numpy()

text_data

array(['голосуем за богу дорогие друзья голосуйте за мой сборник я запущу вас в небеса и книгу мирослава бакулина зубы грешников на книге года как голосовать здесь всё написано',
       'позавчера я написал пост про то как мы заехали за подругой в общежитие тюмгу а её из него не выпустили мне придётся сейчас его продублировать текст ведь оригинал пришлось удалить а оригинал пришлось удалить потому что в нём был упомянут мой знакомый по совместительству помощник местного депутатаединоросса имя называть не буду и хотя человек не принимал участия в вызволении милицы а просто сидел в машине у него изза упоминания возникли проблемы такие дела ребята вместо того чтобы посодействовать по линии депутата в решении вопроса у этой едросистемы аллергия на проявления чувства справедливости смеяться или плакать я сейчас просто просто выпрыгну из окна и всё на первом же решётки нет я не боюсь проблем пусть лучше меня депортируют у нас такое просто невозможно вчера в час ночи я заехал за в общежитие т

In [16]:
text_data_lemmatized = []
for text in log_progress(text_data, every=1):
    text_data_lemmatized.append(preprocess_text(text))
    #print('done')

In [17]:
text_data_lemmatized[0][:10]

['iv',
 'международный',
 'фестиваль',
 'документальный',
 'кино',
 'кинза',
 'тюмень',
 'октябрь',
 'год',
 'i']

In [18]:
data['lines_lemmatized'] = text_data_lemmatized
data.head()

,id,date,owner_id,text,likes,reposts,comments,txt_processed,lines_lemmatized
0,66,2011-12-03 10:18:00,-1243876,"ГОЛОСУЕМ ЗА БОГУ!!\n\n""Дорогие друзья! Голосуй...",6,1,1,голосуем за богу дорогие друзья голосуйте за м...,"[iv, международный, фестиваль, документальный,..."
1,10588,2012-03-19 13:58:45,695803,"Позавчера я написал пост про то, как мы заехал...",8,3,16,позавчера я написал пост про то как мы заехали...,"[позавчера, написать, пост, заезжать, подруга,..."
2,800,2012-04-10 04:55:40,102648080,В эти минуты идет прямая трансляция лекции про...,2,1,0,в эти минуты идет прямая трансляция лекции про...,"[минута, идти, прямой, трансляция, лекция, про..."
3,5810,2012-05-07 06:49:16,70549719,12 апреля 2012 г. в Тюменском государственном ...,1,1,0,апреля г в тюменском государственном университ...,"[апрель, г, тюменский, государственный, универ..."
4,5003,2012-11-12 11:10:00,61866128,"Дорогие друзья, журнал ""ЕГОРКА"" участвует в ко...",2,1,2,дорогие друзья журнал егорка участвует в конку...,"[дорогой, друг, журнал, егорка, участвовать, к..."


In [19]:
data.shape

(56834, 9)

In [20]:
data.tail(10)

,id,date,owner_id,text,likes,reposts,comments,txt_processed,lines_lemmatized
56824,35794,2019-01-13 10:12:30,-66172360,Евгения Басакова: «Рано поверили в победу»\n\n...,0,0,0,евгения басакова рано поверили в победу в очер...,"[евгения, басаков, рано, поверять, победа, оче..."
56825,52647,2019-01-13 08:54:44,-103247095,🔥🔥🔥Maxline ВЕРНЁТ🔥🔥🔥 (Часть 4) \n \nНе сыграла...,81,15,39,maxline вернёт часть не сыграла ставка maxline...,"[maxline, вернуть, часть, сыграть, ставка, max..."
56826,2730,2019-01-13 07:57:56,-77960765,"[id185696918|Татьяна], дипломы и призы для поб...",0,0,0,дипломы и призы для победителей и призеров кон...,"[диплом, приз, победитель, призер, конкурс, по..."
56827,544,2019-01-13 07:52:23,-53025229,"Ребят, сессия.",2,0,0,ребят сессия,"[ребята, сессия]"
56828,440384,2019-01-13 07:47:20,-59042614,"Тот кто Карты нашёл ,если можно оставь пожалуй...",0,0,5,тот кто карты нашёл если можно оставь пожалуйс...,"[карта, находить, оставлять, пожалуйста, любой..."
56829,5579,2019-01-13 07:03:56,-8775943,"Ссылка на трансляцию сегодняшнего матча ""Тюмен...",7,2,1,ссылка на трансляцию сегодняшнего матча тюмень...,"[ссылка, трансляция, сегодняшний, матч, тюмень..."
56830,5578,2019-01-13 06:49:43,-8775943,Евгения Басакова: «Рано поверили в победу» \n\...,9,2,0,евгения басакова рано поверили в победу в очер...,"[евгения, басаков, рано, поверять, победа, оче..."
56831,5575,2019-01-13 06:16:13,-8775943,Анна Поспелова: «Была какая-то паника» \n\n«Пр...,14,1,2,анна поспелова была какаято паника приморочка ...,"[анна, поспелов, какаято, паника, приморочка, ..."
56832,2446,2019-01-13 04:13:14,-174469560,«Приморочка» провела первую в этом году встреч...,0,0,0,приморочка провела первую в этом году встречу ...,"[приморочка, проводить, первый, год, встреча, ..."
56833,35786,2019-01-12 21:11:32,-66172360,Наталья Васильченко: «Нас подвели тонкости пси...,0,0,0,наталья васильченко нас подвели тонкости психо...,"[наталья, васильченко, подводить, тонкость, пс..."


In [21]:
data.to_csv('vk_posts_2011-2020_lemmatized.csv', encoding='utf-8')